Below is the compact mathematical “kernel” you will feed to any solver (Gauss-Newton, IRLS, etc.).

It glues together the two noise sources you modelled in the previous steps.

1. Observation (measurement) equation

For each anchor $i = 1 \dots m$
$$
\boxed{\;
\tilde\theta_i \;=\;
\operatorname{atan2}\!\bigl(y_i - y,\; x_i - x\bigr)
\;+\;
\varepsilon_{\theta i}
\;+\;
\underbrace{\Bigl(\tfrac{\partial\theta_i}{\partial\mathbf A_i}\Bigr)
\!\; \delta\mathbf A_i}_{\text{effect of anchor error}}
\;}
\tag{1}
$$
Unknown state $\mathbf X=(x,y)^{\top}$.\\
Nominal anchor $\mathbf A_i=(x_i,y_i)^{\top}$.\\
Angle noise $\varepsilon_{\theta i}\sim\text{von Mises}(0,\kappa)$ ($\approx \mathcal N(0,\sigma^2_\theta)$ for small $\sigma_\theta$).\\
Anchor noise $\delta\mathbf A_i\sim\mathcal N(\mathbf 0,\Sigma_i)$.\\
Because both noise terms are small, you can move to an orthogonal-distance residual that is linear in the errors and zero at the truth:
$$
r_i(\mathbf X)\;=\;
\cos\tilde\theta_i\,(y - y_i)\;-\;
\sin\tilde\theta_i\,(x - x_i)\;\approx\;0 .
\tag{2}
$$
Let
$\mathbf n_i = \bigl[-\sin\tilde\theta_i,\; \cos\tilde\theta_i\bigr]^{\top},$
$\qquad$
$D_i=\|\mathbf X-\mathbf A_i\|.$
Then $r_i = \mathbf n_i^{\top}(\mathbf X-\mathbf A_i)$.

2. Error (residual) variance

The two independent noise sources propagate into $r_i$ as
$$
\sigma_{r i}^{2}
\;=\;
D_i^{2}\,\sigma_{\theta}^{2}
\;+\;
\mathbf n_i^{\top}\,\Sigma_i\,\mathbf n_i .
\tag{3}
$$
First term: direction error projected over the slant range\\
Second term: anchor-coordinate covariance projected onto the line normal.\\
Dynamic weight
$w_i = 1/\sigma_{r i}^{2}.$

As $D_i$ and $\mathbf n_i$ depend on the current iterate $\mathbf X$, you recompute $w_i$ at every solver step (“total-least-squares weighting”).

3. Weighted loss function

Choose either pure least-squares or a robust M-estimator (Huber).

    \begin{enumerate}
        \item Quadratic (classical ODR/TLS)
        $$
        \boxed{\;
        J_{\text{LS}}(\mathbf X)
        =\frac12
        \sum_{i=1}^{m}
        \frac{r_i(\mathbf X)^{2}}{\sigma_{r i}^{2}}
        \;}
        \tag{4}
        $$

        \item Huber-robust ODR (recommended)
        $$
        \boxed{\;
        J_{\text{Huber}}(\mathbf X)
        =\sum_{i=1}^{m}
        w_i\;
        \rho_H\!\left(\frac{r_i(\mathbf X)}{\sigma_{r i}}\right)
        \;}
        \tag{5}
        $$
        with
        $$
        \rho_H(u)=
        \begin{cases}
        \frac{u^{2}}{2}, & |u|\le\delta,\\[4pt]
        \delta|u|-\frac{\delta^{2}}{2}, & |u|>\delta,
        \end{cases}
        \qquad
        \delta\approx1.5.
        $$
    \end{enumerate}

    \item Normal-equation form for iteration

At each IRLS/Gauss-Newton step compute the Jacobian
$$
\mathbf J =
\begin{bmatrix}
\partial r_1/\partial x & \partial r_1/\partial y\\
\vdots & \vdots\\
\partial r_m/\partial x & \partial r_m/\partial y
\end{bmatrix},
\qquad
\mathbf W=\operatorname{diag}(w_1,\dots,w_m),
$$
and solve
$$
(\mathbf J^{\top}\mathbf W\mathbf J)\,\Delta\mathbf X
=-\mathbf J^{\top}\mathbf W\mathbf r,
\qquad
\mathbf X_{k+1}=\mathbf X_k+\Delta\mathbf X .
$$
Converge when $\|\Delta\mathbf X\|<10^{-5}\,\text{m}$ or iteration limit hit.
\end{enumerate}